#Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#LIVE PHASE 2 Dataset
(separate different distrotion type images into respective folders)

In [ ]:
import os
import shutil

def separate_images_by_distortion(dataset_path, output_path):
    # Create a dictionary for the distortion type folder names
    distortion_folders = {
        '0': 'Reference',
        '1': 'White_Noise',
        '2': 'JPEG_2000',
        '3': 'JPEG',
        '4': 'Gaussian_Blur',
        '5': 'Fast_Fading'
    }

    # Create the main output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Create subdirectories for each distortion type
    for folder_name in distortion_folders.values():
        folder_path = os.path.join(output_path, folder_name)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    # Iterate over the images in the dataset
    for image in os.listdir(dataset_path):
        parts = image.split('_')
        if len(parts) == 3:
            distortion_type = parts[1]
            profile = parts[2][0]  # Removing the '.bmp' extension

            # Determine the appropriate folder
            if profile == '0':
                folder_name = 'Reference'
            else:
                folder_name = distortion_folders.get(distortion_type)

            if folder_name:
                src_path = os.path.join(dataset_path, image)
                dest_path = os.path.join(output_path, folder_name, image)
                shutil.move(src_path, dest_path)

# Define paths
dataset_path = '/content/drive/MyDrive/Phase2/Stimuli'
output_path = '/content/drive/MyDrive/Phase2/Stimuli/separate'

# Separate images into folders based on distortion type and reference
separate_images_by_distortion(dataset_path, output_path)

print("Images have been separated into folders based on distortion type and reference images.")


Images have been separated into folders based on distortion type and reference images.


# Separate left and right images

Description of the input image - the image contains left and right images of a stereo pair.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os
def split_stereo_image(image_path):
  combined_image = cv2.imread(image_path, 0)
  if combined_image is None:
    raise FileNotFoundError(f"Image not found or cannot be read: {image_path}")
  height, width = combined_image.shape
  mid_point = width // 2
  imgL = combined_image[:, :mid_point]
  imgR = combined_image[:, mid_point:]
  return imgL, imgR

#StereoSGBM Algorithm for Disparity Map Generation

In [ ]:
def compute_disparity(imgL, imgR):
    parameters = {
        'minDisparity': 0,
        'numDisparities': 64,
        'blockSize': 8,
        'disp12MaxDiff': 1,
        'uniquenessRatio': 10,
        'speckleWindowSize': 10,
        'speckleRange': 8,
        'P1': 8 * 3 * 8 ** 2,  # 8*number_of_image_channels*SAD_window_size**2
        'P2': 32 * 3 * 8 ** 2  # 32*number_of_image_channels*SAD_window_size**2
    }
    stereo = cv2.StereoSGBM_create(**parameters)
    disp = stereo.compute(imgL, imgR).astype(np.float32)
    disp = cv2.normalize(disp, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    disp = np.uint8(disp)
    return disp

#Disparity Map Generation - Phase 2

In [ ]:
def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            print(f"Skipping non-image file: {image_name}")
            continue

        try:
            imgL, imgR = split_stereo_image(image_path)
            imgL = cv2.resize(imgL, (600, 600))
            imgR = cv2.resize(imgR, (600, 600))
            disp = compute_disparity(imgL, imgR)
            output_path = os.path.join(output_folder, image_name)
            cv2.imwrite(output_path, disp)
            print(f"Saved disparity map for {image_name} to {output_path}")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

def main():
    base_input_path = "/content/drive/MyDrive/Phase2/Stimuli/separate"
    base_output_path = "/content/drive/MyDrive/Phase2/Stimuli/distortedimages"

    folders = [
        "Fast_Fading",
        "Gaussian_Blur",
        "JPEG",
        "JPEG_2000",
        "Reference",
        "White_Noise"
    ]

    for folder in folders:
        input_folder = os.path.join(base_input_path, folder)
        output_folder = os.path.join(base_output_path, folder)
        process_folder(input_folder, output_folder)

if __name__ == '__main__':
    main()


Saved disparity map for 003image_5_2.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_2.bmp
Saved disparity map for 003image_5_3.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_3.bmp
Saved disparity map for 003image_5_4.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_4.bmp
Saved disparity map for 003image_5_6.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_6.bmp
Saved disparity map for 003image_5_1.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_1.bmp
Saved disparity map for 003image_5_8.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_8.bmp
Saved disparity map for 003image_5_5.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/003image_5_5.bmp
Saved disparity map for 003image_5_7.bmp to /content/drive/MyDrive/Phase2/Stimuli/distortedimages/Fast_Fading/0

#Disparity Map Generation - Phase 1

In [ ]:
def process_phase1_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(input_folder):
        if image_name.endswith('_l.bmp'):
            base_name = image_name[:-6]  # Remove '_l.bmp' to get the base name
            left_image_path = os.path.join(input_folder, f"{base_name}_l.bmp")
            right_image_path = os.path.join(input_folder, f"{base_name}_r.bmp")

            if os.path.exists(left_image_path) and os.path.exists(right_image_path):
                try:
                    imgL = cv2.imread(left_image_path, 0)
                    imgR = cv2.imread(right_image_path, 0)

                    if imgL is None or imgR is None:
                        raise FileNotFoundError(f"Images not found or cannot be read: {left_image_path}, {right_image_path}")

                    imgL = cv2.resize(imgL, (600, 600))
                    imgR = cv2.resize(imgR, (600, 600))
                    disp = compute_disparity(imgL, imgR)
                    output_path = os.path.join(output_folder, f"{base_name}.bmp")
                    cv2.imwrite(output_path, disp)
                    print(f"Saved disparity map for {base_name} to {output_path}")
                except Exception as e:
                    print(f"Error processing {left_image_path} and {right_image_path}: {e}")

def main():
    base_input_path_phase1 = "/content/drive/MyDrive/phase1"
    base_output_path_phase1 = "/content/drive/MyDrive/phase1/distortedimages"

    folders = [
        "blur",
        "ff",
        "jp2k",
        "jpeg",
        "refimgs",
        "wn"
    ]

    # Process phase1 folders
    for folder in folders:
        input_folder = os.path.join(base_input_path_phase1, folder)
        output_folder = os.path.join(base_output_path_phase1, folder)
        process_phase1_folder(input_folder, output_folder)

if __name__ == '__main__':
    main()


Saved disparity map for im2_1 to /content/drive/MyDrive/phase1/distortedimages/blur/im2_1.bmp
Saved disparity map for im2_3 to /content/drive/MyDrive/phase1/distortedimages/blur/im2_3.bmp
Saved disparity map for im3_2 to /content/drive/MyDrive/phase1/distortedimages/blur/im3_2.bmp
Saved disparity map for im3_1 to /content/drive/MyDrive/phase1/distortedimages/blur/im3_1.bmp
Saved disparity map for im3_3 to /content/drive/MyDrive/phase1/distortedimages/blur/im3_3.bmp
Saved disparity map for im7_1 to /content/drive/MyDrive/phase1/distortedimages/blur/im7_1.bmp
Saved disparity map for im8_1 to /content/drive/MyDrive/phase1/distortedimages/blur/im8_1.bmp
Saved disparity map for im10_1 to /content/drive/MyDrive/phase1/distortedimages/blur/im10_1.bmp
Saved disparity map for im10_2 to /content/drive/MyDrive/phase1/distortedimages/blur/im10_2.bmp
Saved disparity map for im12_1 to /content/drive/MyDrive/phase1/distortedimages/blur/im12_1.bmp
Saved disparity map for im12_2 to /content/drive/MyDri

#Disparity Map Generation - 3D22MX

In [ ]:
def process_combined_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(input_folder):
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            print(f"Skipping non-image file: {image_name}")
            continue

        image_path = os.path.join(input_folder, image_name)

        try:
            imgL, imgR = split_stereo_image(image_path)
            imgL = cv2.resize(imgL, (600, 600))
            imgR = cv2.resize(imgR, (600, 600))
            disp = compute_disparity(imgL, imgR)
            base_name = os.path.splitext(image_name)[0]  # Get base name without extension
            output_path = os.path.join(output_folder, f"{base_name}.bmp")
            cv2.imwrite(output_path, disp)
            print(f"Saved disparity map for {base_name} to {output_path}")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
def main():
    input_folder = "/content/drive/MyDrive/3DMX22/Images-J2K Noise/Recovered"
    output_folder = "/content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps"
    process_combined_images(input_folder, output_folder)
if __name__ == '__main__':
    main()


Saved disparity map for 01_02 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/01_02.bmp
Saved disparity map for 01_01 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/01_01.bmp
Saved disparity map for 02_01 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/02_01.bmp
Saved disparity map for 01_05 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/01_05.bmp
Saved disparity map for 01_03 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/01_03.bmp
Saved disparity map for 01_04 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/01_04.bmp
Saved disparity map for 03_02 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/03_02.bmp
Saved disparity map for 03_01 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/03_01.bmp
Saved disparity map for 02_05 to /content/drive/MyDrive/3DMX22/Images-J2K Noise/DisparityMaps/02_05.bmp
Saved disparity map for 02_03 to /content/drive/MyDrive/3DMX22/I

#Image score generation using SSIM (Full reference IQA)

#PHASE1

In [ ]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
import pandas as pd

# Define paths
base_path = "/content/drive/MyDrive/phase1"
reference_images_path = os.path.join(base_path, "refimgs")

# List of distortion folders
distortion_folders = ["blur", "ff", "jp2k", "jpeg", "wn"]

# Initialize a list to store results
ssim_scores = []

# List all reference images to debug
reference_images_files = os.listdir(reference_images_path)
print("Reference images found:", reference_images_files)

# Function to compute SSIM
def compute_ssim(distorted_image_path, reference_image_path):
    distorted_image = cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE)
    reference_image = cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE)

    # Check if images are loaded correctly
    if distorted_image is None:
        print(f"Failed to load distorted image: {distorted_image_path}")
        return None
    if reference_image is None:
        print(f"Failed to load reference image: {reference_image_path}")
        return None

    # Compute SSIM
    score, _ = ssim(distorted_image, reference_image, full=True)
    return score

# Iterate over each distortion folder
for folder in distortion_folders:
    distortion_folder_path = os.path.join(base_path, folder)
    distorted_images_files = os.listdir(distortion_folder_path)

    # Iterate over each image in the distortion folder
    for distorted_image_file in distorted_images_files:
        if not distorted_image_file.endswith('.bmp'):
            continue  # Skip non-image files

        # Extract image name and level
        parts = distorted_image_file.split('_')
        base_name = f"{parts[0]}"
        view = parts[-1].split('.')[0]  # Extract 'l' or 'r'

        # Construct the reference image file name
        reference_image_file = f"{base_name}_{view}.bmp"
        reference_image_path = os.path.join(reference_images_path, reference_image_file)

        # Debug statement to show expected reference image
        print(f"Expected reference image: {reference_image_file}")

        # Check if the reference image exists
        if not os.path.exists(reference_image_path):
            print(f"Reference image does not exist: {reference_image_path}")
            continue

        # Construct the distorted image path
        distorted_image_path = os.path.join(distortion_folder_path, distorted_image_file)

        # Compute SSIM
        score = compute_ssim(distorted_image_path, reference_image_path)

        if score is not None:
            # Append the score to the list
            ssim_scores.append((folder, distorted_image_file, score))

# Convert the list to a DataFrame for easy manipulation
ssim_df = pd.DataFrame(ssim_scores, columns=["Distortion_Type", "Image", "SSIM_Score"])

# Save the results to a CSV file
ssim_df.to_csv("ssim_scores.csv", index=False)

print("SSIM scores computed and saved successfully.")


Reference images found: ['im3_r.bmp', 'im2_r.bmp', 'im3_l.bmp', 'im2_l.bmp', 'im5_r.bmp', 'im5_l.bmp', 'im7_r.bmp', 'im8_r.bmp', 'im8_l.bmp', 'im7_l.bmp', 'im10_r.bmp', 'im10_l.bmp', 'im12_l.bmp', 'im12_r.bmp', 'im15_r.bmp', 'im14_l.bmp', 'im13_r.bmp', 'im14_r.bmp', 'im15_l.bmp', 'im13_l.bmp', 'im18_l.bmp', 'im16_r.bmp', 'im18_r.bmp', 'im17_l.bmp', 'im17_r.bmp', 'im16_l.bmp', 'im21_r.bmp', 'im21_l.bmp', 'im20_r.bmp', 'im20_l.bmp', 'im22_r.bmp', 'im22_l.bmp', 'im24_l.bmp', 'im24_r.bmp', 'im27_l.bmp', 'im26_l.bmp', 'im27_r.bmp', 'im26_r.bmp', 'im29_l.bmp', 'im29_r.bmp', 'im2_l_map.mat', 'im3_l_map.mat', 'im3_r_map.mat', 'im2_r_map.mat', 'im5_r_map.mat', 'im5_l_map.mat', 'im8_l_map.mat', 'im7_r_map.mat', 'im8_r_map.mat', 'im7_l_map.mat', 'im10_l_map.mat', 'im10_r_map.mat', 'im12_r_map.mat', 'im12_l_map.mat', 'im13_r_map.mat', 'im14_l_map.mat', 'im15_l_map.mat', 'im13_l_map.mat', 'im15_r_map.mat', 'im14_r_map.mat', 'im17_l_map.mat', 'im16_l_map.mat', 'im16_r_map.mat', 'im17_r_map.mat', 'im

#PHASE2

In [ ]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import pandas as pd

# Define paths
base_path = "/content/drive/MyDrive/Phase2/Stimuli/separate"
reference_images_path = os.path.join(base_path, "Reference")

# List of distortion folders
distortion_folders = ["White_Noise", "JPEG_2000", "JPEG", "Gaussian_Blur", "Fast_Fading"]

# Initialize a list to store results
ssim_scores = []

# Function to compute SSIM
def compute_ssim(image_left, image_right, reference_left, reference_right):
    # Compute SSIM for left and right views
    ssim_left, _ = ssim(image_left, reference_left, full=True)
    ssim_right, _ = ssim(image_right, reference_right, full=True)

    # Average SSIM score
    score = (ssim_left + ssim_right) / 2.0
    return score

# Function to load and split stereo images
def load_and_split_stereo_image(stereo_image_path):
    stereo_image = cv2.imread(stereo_image_path, cv2.IMREAD_GRAYSCALE)

    if stereo_image is None:
        print(f"Failed to load stereo image: {stereo_image_path}")
        return None, None

    # Split the stereo image into left and right views
    height, width = stereo_image.shape
    half_width = width // 2
    image_left = stereo_image[:, :half_width]
    image_right = stereo_image[:, half_width:]

    return image_left, image_right

# Iterate over each distortion folder
for folder in distortion_folders:
    distortion_folder_path = os.path.join(base_path, folder)
    distorted_images_files = os.listdir(distortion_folder_path)

    # Iterate over each stereo image pair in the distortion folder
    for distorted_image_file in distorted_images_files:
        if not distorted_image_file.endswith('.bmp'):
            continue  # Skip non-image files

        # Load and split the distorted stereo image
        stereo_image_path = os.path.join(distortion_folder_path, distorted_image_file)
        image_left, image_right = load_and_split_stereo_image(stereo_image_path)

        if image_left is None or image_right is None:
            continue

        # Extract image number, distortion type, and profile
        parts = distorted_image_file.split('_')
        image_number = parts[0]
        distortion_type = parts[1]
        profile = parts[2].split('.')[0]

        # Determine the reference image file names
        reference_left_file = f"{image_number}_{distortion_type}_0.bmp"
        reference_right_file = f"{image_number}_{distortion_type}_0.bmp"

        # Construct paths for reference left and right images
        reference_left_path = os.path.join(reference_images_path, reference_left_file)
        reference_right_path = os.path.join(reference_images_path, reference_right_file)

        # Load and split the reference stereo images
        reference_left, reference_right = load_and_split_stereo_image(reference_left_path)

        if reference_left is None or reference_right is None:
            continue

        # Compute SSIM score
        score = compute_ssim(image_left, image_right, reference_left, reference_right)

        if score is not None:
            # Append the score to the list
            ssim_scores.append((folder, distorted_image_file, score))

# Convert the list to a DataFrame for easy manipulation
ssim_df = pd.DataFrame(ssim_scores, columns=["Distortion_Type", "Image", "Average_SSIM_Score"])

# Save the results to a CSV file
csv_filename = "ssim_scores_phase2.csv"
ssim_df.to_csv(csv_filename, index=False)

print(f"SSIM scores computed and saved to {csv_filename} successfully.")


SSIM scores computed and saved to ssim_scores_phase2.csv successfully.


#3D22MX

In [ ]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import pandas as pd

# Define paths
base_path = "/content/drive/MyDrive/3DMX22/Images-J2K Noise"
sources_path = os.path.join(base_path, "Sources")
recovered_path = os.path.join(base_path, "Recovered")

# Initialize a list to store results
ssim_scores = []

# Function to compute SSIM
def compute_ssim(image_left, image_right, reference_left, reference_right):
    # Compute SSIM for left and right views
    ssim_left, _ = ssim(image_left, reference_left, full=True)
    ssim_right, _ = ssim(image_right, reference_right, full=True)

    # Average SSIM score
    score = (ssim_left + ssim_right) / 2.0
    return score

# Function to load and split stereo images
def load_and_split_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Failed to load image: {image_path}")
        return None, None

    # Split the image into left and right views
    height, width = image.shape
    half_width = width // 2
    image_left = image[:, :half_width]
    image_right = image[:, half_width:]

    return image_left, image_right

# Iterate over each reference image in the Sources folder
for ref_image_file in os.listdir(sources_path):
    if not ref_image_file.endswith('.bmp'):
        continue  # Skip non-image files

    # Construct paths for reference left and right images
    ref_image_path = os.path.join(sources_path, ref_image_file)
    ref_image_left, ref_image_right = load_and_split_image(ref_image_path)

    if ref_image_left is None or ref_image_right is None:
        continue

    # Determine the base name of the reference image
    ref_image_base = os.path.splitext(ref_image_file)[0]

    # Iterate over distorted images in the Recovered folder
    for recovered_image_file in os.listdir(recovered_path):
        if not recovered_image_file.startswith(ref_image_base):
            continue  # Skip files not related to the current reference image

        # Construct path for the distorted image
        distorted_image_path = os.path.join(recovered_path, recovered_image_file)

        # Load and split the distorted image
        distorted_image_left, distorted_image_right = load_and_split_image(distorted_image_path)

        if distorted_image_left is None or distorted_image_right is None:
            continue

        # Compute SSIM score
        score = compute_ssim(distorted_image_left, distorted_image_right, ref_image_left, ref_image_right)

        if score is not None:
            # Append the score to the list
            ssim_scores.append((ref_image_file, recovered_image_file, score))

# Convert the list to a DataFrame for easy manipulation
ssim_df = pd.DataFrame(ssim_scores, columns=["Reference_Image", "Distorted_Image", "Average_SSIM_Score"])

# Save the results to a CSV file
csv_filename = "ssim_scores_3DMX22.csv"
ssim_df.to_csv(csv_filename, index=False)

print(f"SSIM scores computed and saved to {csv_filename} successfully.")


SSIM scores computed and saved to ssim_scores_3DMX22.csv successfully.


#Module Installation

In [ ]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=189eea527a0bc85330684147ce7f21ccf30f565dca8498de2598146e1b442b26
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [ ]:
pip install pytorch_msssim

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage import img_as_float
from skimage.metrics import mean_squared_error
from math import log10, sqrt
import torch
from pytorch_msssim import ssim as msssim

#Universal Quality Index (UQI)

In [ ]:
def compute_uqi(ref_img, dist_img):
    N = ref_img.size
    ref_img = ref_img.astype(np.float64)
    dist_img = dist_img.astype(np.float64)

    ref_mean = ref_img.mean()
    dist_mean = dist_img.mean()
    ref_var = ref_img.var()
    dist_var = dist_img.var()
    cov = np.mean((ref_img - ref_mean) * (dist_img - dist_mean))

    uqi_score = (4 * ref_mean * dist_mean * cov) / ((ref_mean**2 + dist_mean**2) * (ref_var + dist_var))
    return uqi_score

#MS-SSIM

In [ ]:
def compute_msssim(ref_img, dist_img):
    ref_img = torch.tensor(ref_img).unsqueeze(0).unsqueeze(0).float() / 255.0
    dist_img = torch.tensor(dist_img).unsqueeze(0).unsqueeze(0).float() / 255.0
    return msssim(ref_img, dist_img).item()

#FSIM

In [ ]:
def compute_fsim(ref_img, dist_img):
    ref_img = img_as_float(ref_img)
    dist_img = img_as_float(dist_img)

    # Phase congruency parameters
    epsilon = 1e-12
    T1 = 0.85
    T2 = 160

    # Sobel filters
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    def phase_congruency(img):
        Ix = cv2.filter2D(img, -1, sobel_x)
        Iy = cv2.filter2D(img, -1, sobel_y)
        gradient_magnitude = np.sqrt(Ix**2 + Iy**2)
        return gradient_magnitude

    PC1 = phase_congruency(ref_img)
    PC2 = phase_congruency(dist_img)

    GPC = np.minimum(PC1, PC2) / (np.maximum(PC1, PC2) + epsilon)

    FSIM = np.mean(GPC)

    return FSIM

#GMSD

In [ ]:
def compute_gmsd(distorted_image_path, reference_image_path):
    distorted_image = cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE)
    reference_image = cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE)

    # Check if images are loaded correctly
    if distorted_image is None:
        print(f"Failed to load distorted image: {distorted_image_path}")
        return None
    if reference_image is None:
        print(f"Failed to load reference image: {reference_image_path}")
        return None

    # Convert images to float
    distorted_image = img_as_float(distorted_image)
    reference_image = img_as_float(reference_image)

    # Calculate mean squared error between images
    mse = mean_squared_error(reference_image, distorted_image)

    # Calculate GMSD
    gmsd_score = sqrt(mse)

    return gmsd_score

#SSIM

In [ ]:
def compute_ssim(distorted_image_path, reference_image_path):
    distorted_image = cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE)
    reference_image = cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE)

    # Check if images are loaded correctly
    if distorted_image is None:
        print(f"Failed to load distorted image: {distorted_image_path}")
        return None
    if reference_image is None:
        print(f"Failed to load reference image: {reference_image_path}")
        return None

    # Compute SSIM
    score, _ = ssim(distorted_image, reference_image, full=True)
    return score

#PHASE1

In [ ]:
# Define paths
base_path = "/content/drive/MyDrive/phase1"
reference_images_path = os.path.join(base_path, "refimgs")

# List of distortion folders
distortion_folders = ["blur", "ff", "jp2k", "jpeg", "wn"]

# Initialize a list to store results
metrics_scores = []

# List all reference images to debug
reference_images_files = os.listdir(reference_images_path)
print("Reference images found:", reference_images_files)
# Iterate over each distortion folder
for folder in distortion_folders:
    distortion_folder_path = os.path.join(base_path, folder)
    distorted_images_files = os.listdir(distortion_folder_path)

    # Iterate over each image in the distortion folder
    for distorted_image_file in distorted_images_files:
        if not distorted_image_file.endswith('.bmp'):
            continue  # Skip non-image files

        # Extract image name and level
        parts = distorted_image_file.split('_')
        base_name = f"{parts[0]}"
        view = parts[-1].split('.')[0]  # Extract 'l' or 'r'

        # Construct the reference image file name
        reference_image_file = f"{base_name}_{view}.bmp"
        reference_image_path = os.path.join(reference_images_path, reference_image_file)

        # Debug statement to show expected reference image
        print(f"Expected reference image: {reference_image_file}")

        # Check if the reference image exists
        if not os.path.exists(reference_image_path):
            print(f"Reference image does not exist: {reference_image_path}")
            continue

        # Construct the distorted image path
        distorted_image_path = os.path.join(distortion_folder_path, distorted_image_file)

        # Compute SSIM
        ssim_score = compute_ssim(distorted_image_path, reference_image_path)

        # Compute GMSD
        gmsd_score = compute_gmsd(distorted_image_path, reference_image_path)

        # Compute UQI
        uqi_score = compute_uqi(cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE),
                                cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE))

        # Compute FSIM
        fsim_score = compute_fsim(cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE),
                                  cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE))

        # Compute MS-SSIM
        msssim_score = compute_msssim(cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE),
                                      cv2.imread(distorted_image_path, cv2.IMREAD_GRAYSCALE))

        if all(metric is not None for metric in [ssim_score, gmsd_score, uqi_score, fsim_score, msssim_score]):
            # Append the scores to the list
            metrics_scores.append((folder, base_name, view, ssim_score, gmsd_score, uqi_score, fsim_score, msssim_score))

# Convert the list to a DataFrame for easy manipulation
columns = ["Distortion_Type", "Image", "View", "SSIM_Score", "GMSD_Score", "UQI_Score", "FSIM_Score", "MSSSIM_Score"]
metrics_df = pd.DataFrame(metrics_scores, columns=columns)

# Average the scores for left and right images, ignoring the 'View' column
average_df = metrics_df.groupby(['Distortion_Type', 'Image'], as_index=False).mean(numeric_only=True)

# Save the results to a CSV file
average_df.to_csv("metrics_scores.csv", index=False)

print("Metrics scores computed and saved successfully.")


Reference images found: ['im3_r.bmp', 'im2_r.bmp', 'im3_l.bmp', 'im2_l.bmp', 'im5_r.bmp', 'im5_l.bmp', 'im7_r.bmp', 'im8_r.bmp', 'im8_l.bmp', 'im7_l.bmp', 'im10_r.bmp', 'im10_l.bmp', 'im12_l.bmp', 'im12_r.bmp', 'im15_r.bmp', 'im14_l.bmp', 'im13_r.bmp', 'im14_r.bmp', 'im15_l.bmp', 'im13_l.bmp', 'im18_l.bmp', 'im16_r.bmp', 'im18_r.bmp', 'im17_l.bmp', 'im17_r.bmp', 'im16_l.bmp', 'im21_r.bmp', 'im21_l.bmp', 'im20_r.bmp', 'im20_l.bmp', 'im22_r.bmp', 'im22_l.bmp', 'im24_l.bmp', 'im24_r.bmp', 'im27_l.bmp', 'im26_l.bmp', 'im27_r.bmp', 'im26_r.bmp', 'im29_l.bmp', 'im29_r.bmp', 'im2_l_map.mat', 'im3_l_map.mat', 'im3_r_map.mat', 'im2_r_map.mat', 'im5_r_map.mat', 'im5_l_map.mat', 'im8_l_map.mat', 'im7_r_map.mat', 'im8_r_map.mat', 'im7_l_map.mat', 'im10_l_map.mat', 'im10_r_map.mat', 'im12_r_map.mat', 'im12_l_map.mat', 'im13_r_map.mat', 'im14_l_map.mat', 'im15_l_map.mat', 'im13_l_map.mat', 'im15_r_map.mat', 'im14_r_map.mat', 'im17_l_map.mat', 'im16_l_map.mat', 'im16_r_map.mat', 'im17_r_map.mat', 'im

#PHASE 2

In [ ]:
# Define paths
base_path = "/content/drive/MyDrive/Phase2/Stimuli/separate"
reference_images_path = os.path.join(base_path, "Reference")

# List of distortion folders
distortion_folders = ["White_Noise", "JPEG_2000", "JPEG", "Gaussian_Blur", "Fast_Fading"]

# Initialize a list to store results
metrics_scores = []

# Function to compute SSIM
def compute_ssim(image_left, image_right, reference_left, reference_right):
    ssim_left, _ = ssim(image_left, reference_left, full=True)
    ssim_right, _ = ssim(image_right, reference_right, full=True)
    score = (ssim_left + ssim_right) / 2.0
    return score
# Function to compute GMSD
def compute_gmsd(ref_img, dist_img):
    ref_img = img_as_float(ref_img)
    dist_img = img_as_float(dist_img)

    Ix_ref = cv2.Sobel(ref_img, cv2.CV_64F, 1, 0, ksize=3)
    Iy_ref = cv2.Sobel(ref_img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude_ref = np.sqrt(Ix_ref**2 + Iy_ref**2)

    Ix_dist = cv2.Sobel(dist_img, cv2.CV_64F, 1, 0, ksize=3)
    Iy_dist = cv2.Sobel(dist_img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude_dist = np.sqrt(Ix_dist**2 + Iy_dist**2)

    gm_diff = gradient_magnitude_ref - gradient_magnitude_dist
    gmsd_score = np.std(gm_diff)

    return gmsd_score

# Iterate over each distortion folder
for folder in distortion_folders:
    distortion_folder_path = os.path.join(base_path, folder)
    distorted_images_files = os.listdir(distortion_folder_path)

    # Iterate over each stereo image pair in the distortion folder
    for distorted_image_file in distorted_images_files:
        if not distorted_image_file.endswith('.bmp'):
            continue

        # Load and split the distorted stereo image
        stereo_image_path = os.path.join(distortion_folder_path, distorted_image_file)
        image_left, image_right = split_stereo_image(stereo_image_path)

        if image_left is None or image_right is None:
            continue

        # Extract image number, distortion type, and profile
        parts = distorted_image_file.split('_')
        image_number = parts[0]
        distortion_type = parts[1]
        profile = parts[2].split('.')[0]

        # Determine the reference image file names
        reference_left_file = f"{image_number}_{distortion_type}_0.bmp"
        reference_right_file = f"{image_number}_{distortion_type}_0.bmp"

        # Construct paths for reference left and right images
        reference_left_path = os.path.join(reference_images_path, reference_left_file)
        reference_right_path = os.path.join(reference_images_path, reference_right_file)

        # Load and split the reference stereo images
        reference_left, reference_right = split_stereo_image(reference_left_path)

        if reference_left is None or reference_right is None:
            continue

        # Compute SSIM
        ssim_score = compute_ssim(image_left, image_right, reference_left, reference_right)

        # Compute UQI
        uqi_score = compute_uqi(reference_left, image_left)
        uqi_score_right = compute_uqi(reference_right, image_right)
        avg_uqi_score = (uqi_score + uqi_score_right) / 2.0

        # Compute FSIM
        fsim_score = compute_fsim(reference_left, image_left)
        fsim_score_right = compute_fsim(reference_right, image_right)
        avg_fsim_score = (fsim_score + fsim_score_right) / 2.0

        # Compute GMSD
        gmsd_score = compute_gmsd(reference_left, image_left)
        gmsd_score_right = compute_gmsd(reference_right, image_right)
        avg_gmsd_score = (gmsd_score + gmsd_score_right) / 2.0

        # Compute MS-SSIM
        msssim_score = compute_msssim(reference_left, image_left)
        msssim_score_right = compute_msssim(reference_right, image_right)
        avg_msssim_score = (msssim_score + msssim_score_right) / 2.0

        if all(metric is not None for metric in [ssim_score, avg_uqi_score, avg_fsim_score, avg_gmsd_score, avg_msssim_score]):
            metrics_scores.append((folder, distorted_image_file, ssim_score, avg_uqi_score, avg_fsim_score, avg_gmsd_score, avg_msssim_score))

# Convert the list to a DataFrame for easy manipulation
columns = ["Distortion_Type", "Image", "SSIM_Score", "UQI_Score", "FSIM_Score", "GMSD_Score", "MSSSIM_Score"]
metrics_df = pd.DataFrame(metrics_scores, columns=columns)

# Save the results to a CSV file
csv_filename = "metrics_scores_phase2.csv"
metrics_df.to_csv(csv_filename, index=False)

print(f"Metrics scores computed and saved to {csv_filename} successfully.")


Metrics scores computed and saved to metrics_scores_phase2.csv successfully.


#3D22MX

In [ ]:
# Define paths
base_path = "/content/drive/MyDrive/3DMX22/Images-J2K Noise"
sources_path = os.path.join(base_path, "Sources")
recovered_path = os.path.join(base_path, "Recovered")

# Initialize a list to store results
metrics_scores = []

# Function to compute SSIM
def compute_ssim(image_left, image_right, reference_left, reference_right):
    ssim_left, _ = ssim(image_left, reference_left, full=True)
    ssim_right, _ = ssim(image_right, reference_right, full=True)
    score = (ssim_left + ssim_right) / 2.0
    return score

# Function to compute GMSD
def compute_gmsd(ref_img, dist_img):
    ref_img = img_as_float(ref_img)
    dist_img = img_as_float(dist_img)

    Ix_ref = cv2.Sobel(ref_img, cv2.CV_64F, 1, 0, ksize=3)
    Iy_ref = cv2.Sobel(ref_img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude_ref = np.sqrt(Ix_ref**2 + Iy_ref**2)

    Ix_dist = cv2.Sobel(dist_img, cv2.CV_64F, 1, 0, ksize=3)
    Iy_dist = cv2.Sobel(dist_img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude_dist = np.sqrt(Ix_dist**2 + Iy_dist**2)

    gm_diff = gradient_magnitude_ref - gradient_magnitude_dist
    gmsd_score = np.std(gm_diff)

    return gmsd_score

# Iterate over each reference image in the Sources folder
for ref_image_file in os.listdir(sources_path):
    if not ref_image_file.endswith('.bmp'):
        continue

    # Construct paths for reference left and right images
    ref_image_path = os.path.join(sources_path, ref_image_file)
    ref_image_left, ref_image_right = split_stereo_image(ref_image_path)

    if ref_image_left is None or ref_image_right is None:
        continue

    # Determine the base name of the reference image
    ref_image_base = os.path.splitext(ref_image_file)[0]

    # Iterate over distorted images in the Recovered folder
    for recovered_image_file in os.listdir(recovered_path):
        if not recovered_image_file.startswith(ref_image_base):
            continue

        # Construct path for the distorted image
        distorted_image_path = os.path.join(recovered_path, recovered_image_file)

        # Load and split the distorted image
        distorted_image_left, distorted_image_right = split_stereo_image(distorted_image_path)

        if distorted_image_left is None or distorted_image_right is None:
            continue

        # Compute SSIM score
        ssim_score = compute_ssim(distorted_image_left, distorted_image_right, ref_image_left, ref_image_right)

        # Compute UQI
        uqi_score = compute_uqi(ref_image_left, distorted_image_left)
        uqi_score_right = compute_uqi(ref_image_right, distorted_image_right)
        avg_uqi_score = (uqi_score + uqi_score_right) / 2.0

        # Compute FSIM
        fsim_score = compute_fsim(ref_image_left, distorted_image_left)
        fsim_score_right = compute_fsim(ref_image_right, distorted_image_right)
        avg_fsim_score = (fsim_score + fsim_score_right) / 2.0

        # Compute GMSD
        gmsd_score = compute_gmsd(ref_image_left, distorted_image_left)
        gmsd_score_right = compute_gmsd(ref_image_right, distorted_image_right)
        avg_gmsd_score = (gmsd_score + gmsd_score_right) / 2.0

        # Compute MS-SSIM
        msssim_score = compute_msssim(ref_image_left, distorted_image_left)
        msssim_score_right = compute_msssim(ref_image_right, distorted_image_right)
        avg_msssim_score = (msssim_score + msssim_score_right) / 2.0

        if all(metric is not None for metric in [ssim_score, avg_uqi_score, avg_fsim_score, avg_gmsd_score, avg_msssim_score]):
            metrics_scores.append((ref_image_file, recovered_image_file, ssim_score, avg_uqi_score, avg_fsim_score, avg_gmsd_score, avg_msssim_score))

# Convert the list to a DataFrame for easy manipulation
columns = ["Reference_Image", "Distorted_Image", "SSIM_Score", "UQI_Score", "FSIM_Score", "GMSD_Score", "MSSSIM_Score"]
metrics_df = pd.DataFrame(metrics_scores, columns=columns)

# Save the results to a CSV file
csv_filename = "metrics_scores_3DMX22.csv"
metrics_df.to_csv(csv_filename, index=False)

print(f"Metrics scores computed and saved to {csv_filename} successfully.")


Metrics scores computed and saved to metrics_scores_3DMX22.csv successfully.


#MODEL TRAINING

#PHASE2

In [ ]:
pip install tensorflow

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Constants
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
CHANNELS = 3

# Loading the SSIM scores
ssim_scores = pd.read_csv('/content/drive/MyDrive/metrics_scores_phase2.csv')

# Filtering only SSIM scores and image names
ssim_scores = ssim_scores[['Image', 'SSIM_Score']]

# Split dataset into training and testing sets
def load_dataset(base_path, ssim_scores):
    images = []
    labels = []
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith('.bmp'):
                    file_path = os.path.join(folder_path, file)
                    img = load_img(file_path, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH))
                    img_array = img_to_array(img)
                    images.append(img_array)

                    # Get the corresponding SSIM score, default to 1 for reference images
                    if folder == 'Reference':
                        labels.append(1.0)
                    else:
                        label = ssim_scores[ssim_scores['Image'] == file]['SSIM_Score'].values[0]
                        labels.append(label)
    return np.array(images), np.array(labels)

base_path = '/content/drive/MyDrive/Phase2/Stimuli/separate'
images, labels = load_dataset(base_path, ssim_scores)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Split images into left and right
def split_images(images):
    left_images = images[:, :, :IMAGE_WIDTH//2, :]
    right_images = images[:, :, IMAGE_WIDTH//2:, :]
    return left_images, right_images

train_left, train_right = split_images(train_images)
test_left, test_right = split_images(test_images)

# feature extraction model for left and right images
def build_feature_extractor(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten()
    ])
    return model

input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH//2, CHANNELS)
left_feature_extractor = build_feature_extractor(input_shape)
right_feature_extractor = build_feature_extractor(input_shape)

#combined model
left_input = tf.keras.Input(shape=input_shape)
right_input = tf.keras.Input(shape=input_shape)

left_features = left_feature_extractor(left_input)
right_features = right_feature_extractor(right_input)

combined_features = layers.concatenate([left_features, right_features])
combined_features = layers.Dense(128, activation='relu')(combined_features)
combined_features = layers.Dense(64, activation='relu')(combined_features)
output = layers.Dense(1)(combined_features)

model = tf.keras.Model(inputs=[left_input, right_input], outputs=output)

# Define custom accuracy metric for regression
def regression_accuracy(y_true, y_pred):
    return tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)

model.compile(optimizer='adam', loss='mse', metrics=['mae', regression_accuracy])

# Train the model
history = model.fit([train_left, train_right], train_labels, epochs=20, batch_size=32, validation_data=([test_left, test_right], test_labels))

# Evaluate the model
test_loss, test_mae, test_accuracy = model.evaluate([test_left, test_right], test_labels)
print(f"Test MAE: {test_mae}")
print(f"Test Accuracy: {test_accuracy}")

# Save the model
model.save('/path/to/save/model.h5')


Epoch 1/20
10/10 [==============================] - 19s 2s/step - loss: 925923.5000 - mae: 345.0064 - regression_accuracy: 50287.4492 - val_loss: 77.7244 - val_mae: 8.6456 - val_regression_accuracy: 1149.2390
Epoch 2/20
10/10 [==============================] - 10s 951ms/step - loss: 19.0197 - mae: 3.2225 - regression_accuracy: 453.2437 - val_loss: 0.8017 - val_mae: 0.7989 - val_regression_accuracy: 109.6638
Epoch 3/20
10/10 [==============================] - 11s 1s/step - loss: 0.3139 - mae: 0.4636 - regression_accuracy: 68.7169 - val_loss: 0.1470 - val_mae: 0.3344 - val_regression_accuracy: 41.1611
Epoch 4/20
10/10 [==============================] - 11s 1s/step - loss: 0.0731 - mae: 0.2114 - regression_accuracy: 32.8296 - val_loss: 0.0356 - val_mae: 0.1435 - val_regression_accuracy: 21.3832
Epoch 5/20
10/10 [==============================] - 9s 894ms/step - loss: 0.0431 - mae: 0.1611 - regression_accuracy: 23.3400 - val_loss: 0.0284 - val_mae: 0.1393 - val_regression_accuracy: 18.1992

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr, kendalltau
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Constants
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
CHANNELS = 3

# Loading the SSIM scores
ssim_scores = pd.read_csv('/content/drive/MyDrive/metrics_scores_phase2.csv')

# Filtering only SSIM scores and image names
ssim_scores = ssim_scores[['Image', 'SSIM_Score', 'Distortion_Type']]

# Split dataset into training and testing sets
def load_dataset(base_path, ssim_scores):
    images = []
    labels = []
    distortion_types = []
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith('.bmp'):
                    file_path = os.path.join(folder_path, file)
                    img = load_img(file_path, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH))
                    img_array = img_to_array(img)
                    images.append(img_array)

                    # Get the corresponding SSIM score, default to 1 for reference images
                    if folder == 'Reference':
                        labels.append(1.0)
                        distortion_types.append('Reference')
                    else:
                        label = ssim_scores[ssim_scores['Image'] == file]['SSIM_Score'].values[0]
                        distortion_type = ssim_scores[ssim_scores['Image'] == file]['Distortion_Type'].values[0]
                        labels.append(label)
                        distortion_types.append(distortion_type)
    return np.array(images), np.array(labels), distortion_types

base_path = '/content/drive/MyDrive/Phase2/Stimuli/separate'
images, labels, distortion_types = load_dataset(base_path, ssim_scores)
train_images, test_images, train_labels, test_labels, train_types, test_types = train_test_split(
    images, labels, distortion_types, test_size=0.2, random_state=42)

# Split images into left and right
def split_images(images):
    left_images = images[:, :, :IMAGE_WIDTH//2, :]
    right_images = images[:, :, IMAGE_WIDTH//2:, :]
    return left_images, right_images

train_left, train_right = split_images(train_images)
test_left, test_right = split_images(test_images)

# Feature extraction model for left and right images
def build_feature_extractor(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5)
    ])
    return model

input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH//2, CHANNELS)
left_feature_extractor = build_feature_extractor(input_shape)
right_feature_extractor = build_feature_extractor(input_shape)

# Combined model with additional layers
left_input = tf.keras.Input(shape=input_shape)
right_input = tf.keras.Input(shape=input_shape)

left_features = left_feature_extractor(left_input)
right_features = right_feature_extractor(right_input)

combined_features = layers.concatenate([left_features, right_features])
combined_features = layers.Dense(256, activation='relu')(combined_features)
combined_features = layers.Dropout(0.5)(combined_features)
combined_features = layers.Dense(128, activation='relu')(combined_features)
combined_features = layers.Dropout(0.5)(combined_features)
combined_features = layers.Dense(64, activation='relu')(combined_features)
output = layers.Dense(1)(combined_features)

model = tf.keras.Model(inputs=[left_input, right_input], outputs=output)

# Define custom accuracy metric for regression
def regression_accuracy(y_true, y_pred):
    return tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)

model.compile(optimizer='adam', loss='mse', metrics=['mae', regression_accuracy])

# Train the model
history = model.fit([train_left, train_right], train_labels, epochs=50, batch_size=32, validation_data=([test_left, test_right], test_labels))

# Evaluate the model
test_loss, test_mae, test_accuracy = model.evaluate([test_left, test_right], test_labels)
print(f"Test MAE: {test_mae}")
print(f"Test Accuracy: {test_accuracy}")

# Save the model
model.save('/path/to/save/model.h5')

# Calculate additional metrics
def calculate_metrics(y_true, y_pred):
    plcc, _ = pearsonr(y_true, y_pred)
    srocc, _ = spearmanr(y_true, y_pred)
    krocc, _ = kendalltau(y_true, y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return plcc, srocc, rmse, krocc

# Predict on test data
predictions = model.predict([test_left, test_right]).flatten()

# Calculate and print the metrics for the entire dataset
plcc, srocc, rmse, krocc = calculate_metrics(test_labels, predictions)
print(f"PLCC: {plcc}")
print(f"SROCC: {srocc}")
print(f"RMSE: {rmse}")
print(f"KROCC: {krocc}")

# Calculate and print the metrics for each distortion type
distortion_types_unique = np.unique(test_types)
metrics_by_type = {}

for dtype in distortion_types_unique:
    dtype_indices = [i for i, t in enumerate(test_types) if t == dtype]
    dtype_true = np.array(test_labels)[dtype_indices]
    dtype_pred = predictions[dtype_indices]
    plcc, srocc, rmse, krocc = calculate_metrics(dtype_true, dtype_pred)
    metrics_by_type[dtype] = {
        'PLCC': plcc,
        'SROCC': srocc,
        'RMSE': rmse,
        'KROCC': krocc
    }
    print(f"Metrics for {dtype}:")
    print(f"PLCC: {plcc}")
    print(f"SROCC: {srocc}")
    print(f"RMSE: {rmse}")
    print(f"KROCC: {krocc}")


Epoch 1/50
10/10 [==============================] - 20s 2s/step - loss: 11.4940 - mae: 2.4881 - regression_accuracy: 348.9391 - val_loss: 8.7372 - val_mae: 2.4365 - val_regression_accuracy: 311.4955
Epoch 2/50
10/10 [==============================] - 15s 1s/step - loss: 8.1144 - mae: 1.8913 - regression_accuracy: 264.1525 - val_loss: 3.0815 - val_mae: 1.3726 - val_regression_accuracy: 187.1042
Epoch 3/50
10/10 [==============================] - 15s 2s/step - loss: 6.8856 - mae: 1.6787 - regression_accuracy: 241.8582 - val_loss: 33.0415 - val_mae: 5.2977 - val_regression_accuracy: 693.1536
Epoch 4/50
10/10 [==============================] - 16s 2s/step - loss: 5.3936 - mae: 1.4696 - regression_accuracy: 208.7765 - val_loss: 1.3461 - val_mae: 0.9297 - val_regression_accuracy: 118.3487
Epoch 5/50
10/10 [==============================] - 14s 1s/step - loss: 5.1456 - mae: 1.3372 - regression_accuracy: 191.4402 - val_loss: 2.2273 - val_mae: 1.1816 - val_regression_accuracy: 152.3513
Epoch 6/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - 1s 289ms/step
PLCC: 0.36785254435430115
SROCC: 0.28884860496652154
RMSE: 0.2045731053055107
KROCC: 0.21783275336660088
Metrics for Fast_Fading:
PLCC: 0.43336251018420907
SROCC: 0.3558823529411765
RMSE: 0.20782193718658648
KROCC: 0.21666666666666667
Metrics for Gaussian_Blur:
PLCC: 0.5268984259202909
SROCC: 0.7363636363636363
RMSE: 0.1326126864659749
KROCC: 0.6363636363636364
Metrics for JPEG:
PLCC: -0.07023933577617456
SROCC: -0.3939393939393939
RMSE: 0.1901816154103675
KROCC: -0.28888888888888886
Metrics for JPEG_2000:
PLCC: 0.6576353437650337
SROCC: 0.6642857142857141
RMSE: 0.1784206279649963
KROCC: 0.48571428571428577
Metrics for Reference:
PLCC: nan
SROCC: nan
RMSE: 0.35721177871263093
KROCC: nan
Metrics for White_Noise:
PLCC: 0.32326468186966517
SROCC: 0.2507739938080495
RMSE: 0.139109053351779
KROCC: 0.15032679738562094


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
